In [2]:
from typing import List, Annotated
from langchain_google_vertexai import VertexAI
import os
import json
from google.cloud import aiplatform
from autogen_core.tools import FunctionTool
from pathlib import Path

# Initialize Vertex AI
CREDENTIALS_PATH = "../credentials/bwa-agents-ad0e1f5ab4b7.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(CREDENTIALS_PATH)
aiplatform.init(project='bwa-agents', location='us-central1')


def decompose_question(
    question: Annotated[str, "Complex question to decompose"]
) -> List[Annotated[str, "Simpler sub-question"]]:
    """
    Tool: DecomposeQuestion – Splits a complex question into two simpler sub-questions.
    """
    llm = VertexAI(model_name="gemini-2.0-flash-lite-001")
    prompt = (
        f"Decompose the following complex question into exactly two simpler, self-contained sub-questions. "
        f"The sub-questions, when answered sequentially, should help answer the original complex question. "
        f"Return the two sub-questions as a array of strings.\n\n"
        f"Complex Question: \"What is the Köppen climate classification for the city where this mosque is located?\"\n"
        f"Output: [\"In which city is this mosque located?\", \"What is the Köppen climate classification for this city?\"]\n\n"
        f"Complex Question: \"Who is the CEO of the company that developed the game featuring a plumber who jumps on turtles?\"\n"
        f"Output: [\"Which company developed the game featuring a plumber who jumps on turtles?\", \"Who is the CEO of that company?\"]\n\n"
        f"Complex Question: \"{question}\"\n"
        f"Output:"
    )
    resp = llm.invoke(prompt)
    try:
        # Expecting JSON array
        result = json.loads(resp)
        if isinstance(result, list) and len(result) == 2:
            return result  # type: ignore
    except json.JSONDecodeError:
        pass
    # Fallback: take first two non-empty lines
    lines = [ln.strip(' -"') for ln in resp.splitlines() if ln.strip()]
    return lines[:2]


examples = [
    "What is the tallest mountain in the country where the world’s largest waterfall is located?",
    "Who is the author of the novel that inspired the movie about a girl who shrinks to the size of an ant?",
    "How many goals did the top scorer of the 2022 World Cup score, and which country did he play for?"
]

for idx, q in enumerate(examples, 1):
    print(f"\nExample #{idx}:")
    print("Original question:", q)
    try:
        subs = decompose_question(q)
        print("Sub-questions:")
        for i, sub in enumerate(subs, 1):
            print(f"  {i}. {sub}")
    except Exception as e:
        print("Error calling decompose_question:", e)


Example #1:
Original question: What is the tallest mountain in the country where the world’s largest waterfall is located?
Sub-questions:
  1. In which country is the world’s largest waterfall located?
  2. What is the tallest mountain in that country?

Example #2:
Original question: Who is the author of the novel that inspired the movie about a girl who shrinks to the size of an ant?
Sub-questions:
  1. Which novel inspired the movie about a girl who shrinks to the size of an ant?
  2. Who is the author of that novel?

Example #3:
Original question: How many goals did the top scorer of the 2022 World Cup score, and which country did he play for?
Sub-questions:
  1. Who was the top scorer of the 2022 World Cup?
  2. How many goals did that player score, and which country did they play for?
